In [4]:
import numpy as np
import pandas as pd
import os
import  chardet
print(os.getcwd())
import matplotlib.pyplot as plt

distributions = ['DIST_long', 'DIST_short']
leagues = ['ENG1','GER1']
seasons = ['2017_2018','2018_2019']

def accuracy_indicator_function():
    max_indices_f_prob = np.argmax(f_prob, axis=1) # get indices of maximum
    max_indices_o_true = np.argmax(o_true, axis=1) # get indices of maximum
    # xor = np.logical_xor(max_indices_f_prob, max_indices_o_true)
    # not_xor = np.logical_not(xor).astype(int)
    # return not_xor
    list = []
    for i in range(0,len(max_indices_o_true)):
        if max_indices_f_prob[i] == max_indices_o_true[i]:
            list.append(1)
        else:
            list.append(0)
    return np.array(list)

import re


experiments = ['10 Window Experiments','15 Window Experiments','20 Window Experiments','ELO Model Experiments']
tournament_col = []
for experiment in experiments:
    print(experiment)

    dist_col = []
    league_col = []
    rps_col = []
    brier_col = []
    acc_col = []
    simple_col = []
    kelly_col = []
    data_eng_tournament = {}
    data_ger_tournament = {}
    # i = 0
    # j=0
    for league in leagues:
        for dist in distributions:
            for season in seasons:
                # keywords = f'{dist}&{league}&{season}'
                filename = ""
                directory =f"./{experiment}"
                # print("moli")
                # print(os.listdir(directory))
                for name in os.listdir(directory):
                # load the data from the .npz file
                #     print(name)
                    dist_index = name.find(dist)
                    league_index = name.find(league)
                    season_index = name.find(season)
                    # if re.search(keywords,name):
                    #     print(keywords, "found", name)
                    # print(dist_index,league_index,season_index)
                    if dist_index > 0 and league_index > 0 and season_index > 0:
                        filename = f"{directory}/{name}"
                        loaded_data = np.load(filename, allow_pickle=True)

                        # Extract the arrays from the loaded data
                        team_names = loaded_data["team_names"]
                        expected_points = loaded_data["expected_points"]
                        all_points = loaded_data["all_points"]
                        # current_points = loaded_data["c"]
                        f_HDA_array = loaded_data["f_HDA"]
                        f_HDA_year_range_array = loaded_data["f_HDA_year_range"]
                        f_HDA_metadata_array = loaded_data["f_HDA_metadata"]
                        loaded_score_log = loaded_data["score_log"]
                        YYYY_range_file_names = loaded_data["YYYY_range_file_names"]
                        frequencies_home_win = loaded_data["frequencies_home_win"]
                        frequencies_home_draw = loaded_data["frequencies_home_draw"]
                        frequencies_home_loss = loaded_data["frequencies_home_loss"]
                        simulated_simple_balance = loaded_data["simulated_simple_balance"]
                        simulated_kelly_balance = loaded_data["simulated_kelly_balance"]
                        brier_score_and_rps_avg = loaded_data["brier_score_and_rps_avg"]
                        rps_all = loaded_data["rps_all"]
                        o_true = loaded_data["o_true"]
                        f_prob = loaded_data["f_prob"]
                        country_data = loaded_data["country_data"]
                        level_data = loaded_data["level_data"]

                        # get original dictionaries
                        f_HDA = { 'H': f_HDA_array[0], 'D': f_HDA_array[1], 'A': f_HDA_array[2] }
                        f_HDA_year_range = { 'YYYY_start': f_HDA_year_range_array[0], 'YYYY_end': f_HDA_year_range_array[1] }
                        f_HDA_metadata = { 'Countries': f_HDA_metadata_array[0], 'Level': f_HDA_metadata_array[1] }

                        # get ordered points dataframe
                        team_names = team_names.tolist()
                        expected_points = expected_points.tolist()
                        points_df = pd.DataFrame(np.array([team_names,expected_points]).T, columns=["Team","Simulated Average Points"])
                        points_df = points_df.sort_values(by=["Simulated Average Points"], ascending=False, ignore_index=True)
                        points_df.index += 1
                        brier_score = brier_score_and_rps_avg[0]

                        rps_avg = brier_score_and_rps_avg[1]
                        Country = country_data[0]
                        Level = level_data[0]
                        YYYY_start = YYYY_range_file_names[0]
                        YYYY_end = YYYY_range_file_names[1]
                        k = len(simulated_simple_balance)

                        indicator_list = accuracy_indicator_function()
                        accuracy = np.sum(indicator_list)/len(indicator_list)


                        all_points = all_points.tolist()

                        mean_balance_simple = (np.mean(np.array(simulated_simple_balance[:,[-1]])))
                        mean_balance_kelly = (np.mean(np.array(simulated_kelly_balance[:,[-1]])))
                        initial_balance = len(f_prob) * 10
                        # print(initial_balance)
                        profit_simple = mean_balance_simple - initial_balance
                        profit_kelly = mean_balance_kelly - initial_balance

                        dist_col.append(dist)
                        league_col.append(league)
                        rps_col.append(round(rps_avg,4))
                        brier_col.append(round(brier_score,4))
                        acc_col.append(round(accuracy,4))
                        simple_col.append(round(profit_simple))
                        kelly_col.append(round(profit_kelly))

                        team_names = np.array(points_df['Team'].T)
                        team_points = np.array(points_df['Simulated Average Points'].T).astype(float)

                        # print(team_points)
                        team_points = np.round(team_points, 2)
                        points_list = [f' ({point})' for point in team_points ]
                        team_points = np.array(points_list)
                        team_name_and_points = team_names+team_points
                        print(team_name_and_points)
                        # tournament_col.append(team_name_and_points)
                        # print(team_name_and_points.tolist())
                        if league == 'ENG1':
                            data_eng_tournament[f'{dist} {league} {season}'] = team_name_and_points
                            # i+=1
                        else:
                            data_ger_tournament[f'{dist} {league} {season}']=team_name_and_points.tolist()
                            # j+=1

    data_metrics = {
        'Distribution': dist_col,
        'Predicted League': league_col,
        'RPS_avg': rps_col,
        'Brier': brier_col,
        'Acc': acc_col,
        'Profit_simple':simple_col,
        'Profit_kelly':kelly_col
    }


    df_metrics = pd.DataFrame(data_metrics)
    # df_metrics['Profit_simple'] = df_metrics['Profit_simple'].apply(lambda x: "€{:}".format(x))
    # df_metrics['Profit_kelly'] = df_metrics['Profit_kelly'].apply(lambda x: "€{:}".format(x))

    df_eng_tournaments = pd.DataFrame(data_eng_tournament)
    df_ger_tournaments = pd.DataFrame(data_ger_tournament)

    df_metrics.to_csv(f"Export/{experiment}_metrics.csv")
    df_ger_tournaments.to_csv(f"Export/{experiment}_eng_tournament.csv")
    df_eng_tournaments.to_csv(f"Export/{experiment}_ger_tournament.csv")
            # print(dist, league, season, rps_avg, brier_score, accuracy, profit_simple, profit_kelly)

C:\Users\steve\ict3913_stevegalea\models\evaluate\FYP
10 Window Experiments
['Man City (60.56)' 'Chelsea (59.88)' 'Arsenal (59.53)' 'Tottenham (59.0)'
 'Liverpool (56.26)' 'Swansea (56.14)' 'Southampton (52.94)'
 'Man United (52.66)' 'West Ham (51.65)' 'Bournemouth (51.33)'
 'Leicester (49.21)' 'Stoke (49.09)' 'Newcastle (48.86)' 'Everton (48.61)'
 'Brighton (48.56)' 'Crystal Palace (48.52)' 'West Brom (48.43)'
 'Burnley (47.93)' 'Huddersfield (47.55)' 'Watford (46.89)']
['Man City (61.56)' 'Crystal Palace (59.07)' 'Man United (58.51)'
 'Tottenham (56.57)' 'Arsenal (55.2)' 'Everton (54.29)' 'Chelsea (54.2)'
 'Liverpool (53.29)' 'Bournemouth (52.04)' 'Burnley (51.47)'
 'West Ham (50.6)' 'Newcastle (50.54)' 'Leicester (50.38)'
 'Southampton (49.29)' 'Huddersfield (48.74)' 'Cardiff (48.17)'
 'Wolves (47.59)' 'Watford (47.58)' 'Brighton (47.47)' 'Fulham (47.32)']
['Tottenham (59.53)' 'Chelsea (59.35)' 'Arsenal (58.46)'
 'Man City (57.77)' 'Liverpool (56.67)' 'Swansea (53.7)'
 'Man United (